In [7]:
"""[summary]
The script is to compute the cosine simliarities between queries and the document
The steps are as follows:
1. read the tf file
2. obtaint the input query
3. compute the cosine simliarities between query and each document
4.sort by the one with highest simliarity with the highest rank

"""

'[summary]\nThe script is to compute the cosine simliarities between queries and the document\nThe steps are as follows:\n1. read the tf file\n2. obtaint the input query\n3. compute the cosine simliarities between query and each document\n4.sort by the one with highest simliarity with the highest rank\n\n'

In [8]:
def readFile(_filePath):
    """[summary]

    Args:
        _filePath ([type]): [description]
    """
    import pandas as pd
    readFile = pd.read_csv(_filePath,index_col=0)
    return(readFile)

In [9]:
def queryVector(query,corpus):
    """[summary]
    vectorize query based on corpus
    
    Args:
        query ([list]): [list containing query terms]
    """
    corpusList = list(corpus.index)
    ## vector to track the corpusList
    vector = [0]*len(corpusList)
    ## format query\r\n",
    ## import scripts to process queries\r\n"
    import utils.query_processing
    ## convert to string if not string\r\n"
    if type(query) != str:
        query = ' '.join(query)
    ## clean the query
    query = utils.query_processing.process_query(query)

    "still lacking the query expansion"
    "Need  glove_kv and topn to be saved in the repo"
    # query = utils.query_processing.expand_query(query)    
    
    ## iterate over query terms
    for wordOfQuery in query:
    ## variable to tract the corpus index
        vectorIndex = 0
    ## iterate over the all corupus
        for wordOfCorpus in corpusList:
            if wordOfQuery ==  wordOfCorpus:
                vector[vectorIndex] += 1 
    ## update courpus index
            vectorIndex += 1               
    return(vector)

In [10]:
def cosineSimilarities(queryVector,docVector):
    "    \"\"\"[summary]\r\n"
    "    To compute the cosine similarity of the query with a single column/module in corpus\r\n"
    "    Args:\r\n"
    "        queryVector ([list]): [query]\r\n"
    "        docVector ([list]): [description]\r\n"
    from scipy import spatial
    distance = 1 - spatial.distance.cosine(queryVector, docVector)
    return(round(distance,5))

In [11]:
def topModulesForEachQuery(_df):
    "[summary]\r\n",
    "    To obtain the top modules for each of the query\r\n"
    "    Args:\r\n"
    "        _df ([dataframe]): [description] data with rows of words and columns contain the modules, the cell of the dataframe is the similarity\r\n"
    overallTopModules = {}
    for query in _df.columns:
        singleModule = {}
        df.sort_values(by = query,ascending = False, inplace = True)
        ## obtain top 10 modules
        topModules = df[query][:10]
        singleModule['topModules'] = list(topModules.index)
        singleModule['topModulesScore'] = list(topModules)
        overallTopModules[str(query)] = (singleModule)
        
    import pandas as pd
    overallTopModulesDf = pd.DataFrame(overallTopModules)
    return(overallTopModulesDf)

In [12]:
if __name__ == "__main__":
    import pandas as pd
    corpus = readFile('../data/course_info_scores/course_info_tf.csv') ## <-- assign the merge corupus here
    # corpus = readFile('../data/course_info_with_survey_scores/course_info_with_survey_tf.csv') ## <-- assign the merge corupus here
    
    ## vaildation set
    """[summary]
    U can contorl which vaildation set to choose to test the cosine similarity model
    True : would be for the testingQuery that is self create which ensure a value when computing cosineSimilarities
    False : obtain the query based on 50% of vaildation dataset which might have all cosine similarities of zero
    """
    if False:
        testingQuery1 = ['ability','able','abstract']
        testingQuery2 = ['abstraction','accommodate','accompanies']
        testingQuery3 = ['account','accounting','achieve']
        testingQueryList = [[testingQuery1],[testingQuery2],[testingQuery3]]
        testingQueryDf = pd.DataFrame.from_records(testingQueryList,columns = ['querySample'])
        queries = testingQueryDf
        
    else:
        vaildationQueries = readFile('../data/survey/vaildation_sample_query.csv')
        queries = vaildationQueries
    
    ## iterate over the vaildationQueries
    queriesScore = {}
    for row,query in queries.T.iteritems():
        print('\nCurrent computing Query: {}'.format(query['querySample']))
        singleQueryVector = queryVector(query['querySample'],corpus)
        print("Number of terms in corpus: {}".format(sum(singleQueryVector)))
        
        CosineSimilaritiesScore = []
        for modules in list(corpus):
            SimilaritiesScore = cosineSimilarities(singleQueryVector,corpus[modules])
            CosineSimilaritiesScore.append(SimilaritiesScore)
            # print("Score for Query on {} : {}".format(modules,SimilaritiesScore))
        queriesScore[str(query['querySample'])] = CosineSimilaritiesScore
        
    ## Convert to dict to df
    modules = list(corpus)
    df  = pd.DataFrame.from_dict(queriesScore,orient='index',columns=modules)
    df = df.T
    
    ##obtaining top modules for each query
    rankedModule = topModulesForEachQuery(df)
    rankedModule



Current computing Query: network, term, model, technology, probability


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Number of terms in corpus: 5

Current computing Query: term, different, skill, mongodb, long
Number of terms in corpus: 4

Current computing Query: logistics, analysis, operation, basic, r
Number of terms in corpus: 4

Current computing Query: esd, network, evaluate, program, r
Number of terms in corpus: 4

Current computing Query: operation, basic, evaluation, price, evaluate
Number of terms in corpus: 5

Current computing Query: infrastructure, metric, pytorch, model, client
Number of terms in corpus: 5

Current computing Query: approach, logistics, shag, infrastructure, equity
Number of terms in corpus: 4

Current computing Query: data, sql, different, analytics, model
Number of terms in corpus: 5

Current computing Query: computational, best, server, certain, ec
Number of terms in corpus: 4

Current computing Query: sklearn, metric, demand, schedule, fundamental
Number of terms in corpus: 5

Current computing Query: digitalisation, skill, long, technology, aviation
Number of terms 

C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\spatial\distance.py:728: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)



Current computing Query: financial, spark, science, focus, search
Number of terms in corpus: 5

Current computing Query: supply, notebook, fundamental, know, basic
Number of terms in corpus: 4

Current computing Query: r, linear, ethereum, c, machine
Number of terms in corpus: 3

Current computing Query: science, future, problem, demand, pytorch
Number of terms in corpus: 5

Current computing Query: r, ec, kera, opponent, model
Number of terms in corpus: 2

Current computing Query: markov, descent, algebra, math, research
Number of terms in corpus: 4

Current computing Query: clojurescript, ai, analysis, background, science
Number of terms in corpus: 4

Current computing Query: optimisation, opencv, risk, chain, urban
Number of terms in corpus: 5

Current computing Query: maing, descent, science, clojurescript, knowledge
Number of terms in corpus: 3

Current computing Query: schedule, gradient, airport, analysis, system
Number of terms in corpus: 4

Current computing Query: logistics,

In [13]:
rankedModule

,"network, term, model, technology, probability","term, different, skill, mongodb, long","logistics, analysis, operation, basic, r","esd, network, evaluate, program, r","operation, basic, evaluation, price, evaluate","infrastructure, metric, pytorch, model, client","approach, logistics, shag, infrastructure, equity","data, sql, different, analytics, model","computational, best, server, certain, ec","sklearn, metric, demand, schedule, fundamental",...,"r, linear, ethereum, c, machine","science, future, problem, demand, pytorch","r, ec, kera, opponent, model","markov, descent, algebra, math, research","clojurescript, ai, analysis, background, science","optimisation, opencv, risk, chain, urban","maing, descent, science, clojurescript, knowledge","schedule, gradient, airport, analysis, system","logistics, business, strategy, analytics, math","program, future, kera, ec, concept"
topModules,"[50.012 Networks, 01.117 Brain-Inspired Comput...","[40.240 Investment Science, 40.232 Water Resou...","[40.320 Airport Systems Planning and Design, 0...","[50.012 Networks, 50.035 Computer Vision, 50.0...","[50.045 Information Retrieval, 40.320 Airport ...","[40.232 Water Resources Management, 01.117 Bra...","[40.323 Equity Valuation, 40.318 Supply Chain ...","[50.038 Computational Data Science, 50.043 Dat...","[50.048 Computational Fabrication, 40.317 Fina...","[40.302 Advanced Topics in Optimisation#, 40.3...",...,"[50.007 Machine Learning, 40.319 Statistical a...","[50.021 Artificial Intelligence, 40.302 Advanc...","[40.232 Water Resources Management, 01.117 Bra...","[50.048 Computational Fabrication, 40.305 Adva...","[01.116 AI for Healthcare (Term 7), 50.038 Com...","[40.260 Supply Chain Management, 40.318 Supply...","[50.038 Computational Data Science, 50.021 Art...",[40.321 Airport Systems Modelling and Simulati...,[40.318 Supply Chain Digitalisation and Design...,[40.305 Advanced Topics in Stochastic Modellin...
topModulesScore,"[0.38203, 0.32349, 0.2742, 0.2699, 0.26124, 0....","[0.13704, 0.12263, 0.09435, 0.08824, 0.08805, ...","[0.21958, 0.13954, 0.13616, 0.11361, 0.10863, ...","[0.42712, 0.2516, 0.24957, 0.19829, 0.16388, 0...","[0.15969, 0.15275, 0.12497, 0.12481, 0.10243, ...","[0.21936, 0.19907, 0.18685, 0.1681, 0.16663, 0...","[0.08607, 0.0851, 0.04402, 0.03467, 0.03305, 0...","[0.41958, 0.29751, 0.27372, 0.24678, 0.21776, ...","[0.23492, 0.05726, 0.04746, 0.02799, 0.01893, ...","[0.14261, 0.08729, 0.08464, 0.05872, 0.04858, ...",...,"[0.21668, 0.13401, 0.12897, 0.1201, 0.05243, 0...","[0.21173, 0.14261, 0.11988, 0.11349, 0.09716, ...","[0.34684, 0.31476, 0.29544, 0.26578, 0.26347, ...","[0.04698, 0.04508, 0.04181, 0.03753, 0.03426, ...","[0.27492, 0.13403, 0.08607, 0.05564, 0.05463, ...","[0.2186, 0.16746, 0.11885, 0.10968, 0.08332, 0...","[0.10318, 0.0841, 0.05622, 0.04884, 0.0354, 0....","[0.29318, 0.21958, 0.15945, 0.08607, 0.05463, ...","[0.22835, 0.18746, 0.08065, 0.04061, 0.03176, ...","[0.13525, 0.12544, 0.11785, 0.07972, 0.07538, ..."
